In [ ]:

# -*- coding: utf-8 -*-
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.instruments as instruments

import json
import datetime
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

import time

get_ipython().run_line_magic('matplotlib', 'inline')
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
import pickle
import math
import glob
import talib as ta
import shutil
import os

import schedule
from tqdm.notebook import tqdm
from sklearn.model_selection import GridSearchCV
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
from pprint import pprint

#自分のアカウント、トークンをセット
accountID = "101-009-16415310-001"
access_token = '860c2d2dbbcd01b6d95939b6af6b5981-8664724ce0870220c55161d704137df6'

api = API(access_token=access_token, environment="practice")
        
# Oandaからcandleデータを取得する。
def getCandleDataFromOanda(instrument, api, date_from, date_to, granularity):
    params = {
        "from": date_from.isoformat(),
        "to": date_to.isoformat(),
        "granularity": granularity,
    }
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    return api.request(r)

def oandaJsonToPythonList(JSONRes):
    
    data = []
    for res in JSONRes['candles']:
        temp = [((datetime.datetime.fromisoformat(res['time'][:19]))+datetime.timedelta(hours=+9)).strftime('%Y-%m-%d %H:%M')]#indexとなる時間を追加
        temp.extend([res['volume'],
            res['mid']['o'],
            res['mid']['h'],
            res['mid']['l'],
            res['mid']['c'],
            ])
        data.append(temp)
    return data

def logarism(x):
    x = abs(x)
    return math.log(x)


def get_now_data(time, period_days, money, foot, back_days = 0):
    
    all_data = []
    #time = time.replace(hour=7, minute=0, second=0, microsecond=0)
    #入力された日の当日の7時
    
    global PAST
    past = datetime.timedelta(hours=-PAST)
    
    NY = datetime.timedelta(hours=-14)
    back = datetime.timedelta(days=-back_days)
        
    date_to = time + NY + datetime.timedelta(seconds=-1) + back + past

    date_from = date_to + datetime.timedelta(days=-period_days)
    
    ret = getCandleDataFromOanda(money, api, date_from, date_to, foot)
    month_data = oandaJsonToPythonList(ret)#取得したデータを格納

    all_data.extend(month_data)#データを追加

    # pandas DataFrameへ変換
    df = pd.DataFrame(all_data)

    df.columns = ['Datetime','Volume', 'Open', 'High', 'Low', 'Close']
    
    df['Volume'] = df['Volume'].astype('double')
    df['Open'] = df['Open'].astype('double')
    df['Close'] = df['Close'].astype('double')
    
    
    close = np.array(df["Close"]).astype(float)
    #以下、talibを用いてテクニカル指標（今回の学習で用いる特徴量）を算出しdf_feature入れる
    #単純移動平均は、単純移動平均値とその日の終値の比を特徴量として用いる
    df["SMA5"]= ta.SMA(close, timeperiod=5) / close
    df["SMA10"]= ta.SMA(close, timeperiod=10) / close
    #df["RSI"] = ta.RSI(close, timeperiod=9) / close
    #ボリンジャーバンド 
    upper, middle, lower = ta.BBANDS(close, timeperiod=10, nbdevup=2, nbdevdn=2)
    df["BBANDS+σ"] = upper / close
    df["BBANDS-σ"] = lower / close
    
    #df['Rate']=(df['Close']-df['Open'])/df['Open']
    
    #df['Open'] = df['Open'].apply(lambda x: logarism(x))
    #df['Close'] = df['Close'].apply(lambda x: logarism(x))
    
    df['Rate']=(df['Close'].astype('double')-df['Open'].astype('double'))/df['Open'].astype('double')
    
    count = [0,0]
    df['class'] = (df["Rate"]).apply(lambda x: classify_two(x, count))
    
    df['Rate'] = df['Rate']
    
    df = df.dropna()
    
#    df = df.drop('Open', axis=1)
#    df = df.drop('High', axis=1)
#    df = df.drop('Low', axis=1)
#    df = df.drop('Close', axis=1)
#     df = df.drop('Volume', axis=1)
#     df = df.drop('Rate', axis=1)
    
#     df = df.drop('SMA5', axis=1)
#     df = df.drop('SMA10', axis=1)
#     df = df.drop('RSI', axis=1)
#     df = df.drop('BBANDS+σ', axis=1)
#     df = df.drop('BBANDS-σ', axis=1)
    
    
    
    
    df = df.set_index('Datetime')
    
    
    return df



def get_csv(back_days, days, money):
    
    time = datetime.datetime.now() 
    
    df = get_now_data(time, period_days=days, money=money, foot = FOOT , back_days=back_days)
    date_to = (time+datetime.timedelta(days=-back_days)+datetime.timedelta(seconds=-1)).strftime("%Y-%m-%d")
    date_from = (time + datetime.timedelta(days=-days)+datetime.timedelta(days=-back_days)).strftime("%Y-%m-%d")
    df.to_csv("csv/"+str(money)+"_"+ date_from + "_" + date_to +"_"+str(days)+'.csv', mode = 'w')
    print('"'+str(money)+"_"+ date_from + "_" + date_to +"_"+str(days)+'.csv', end='", ')
    
    return df

def get_moneys_csv(back_days, days, moneylist):
    csvlist = []
    
    time = datetime.datetime.now()
    date_to = (time+datetime.timedelta(days=-back_days)+datetime.timedelta(seconds=-1)).strftime("%Y-%m-%d")
    date_from = (time + datetime.timedelta(days=-days)+datetime.timedelta(days=-back_days)).strftime("%Y-%m-%d")
    
    for i in range (len(moneylist)):
        get_csv(back_days, days, money=moneylist[i])
        csvlist.extend([str(moneylist[i])+"_"+ date_from + "_" + date_to +"_"+str(days)+'.csv'])
    
    return csvlist
        
def classify_two(x, c):
        R=0.01
        
        if x<R:
            c[0]+=1
            return -1
        elif R<=x:
            c[1]+=1
            return 1
        

def predict(dataname, moneyname, test_days, back_day, datanum):

    df = pd.read_csv("csv/"+dataname, index_col='Datetime')
    
#     count = [0,0]
    #df['class'] = (df["Rate"]).apply(lambda x: classify(x, count, 0.001))
    df['class'] = df['class'].shift(-1)
    print("Train Data")
    print(" LOW/HIGH")
#     print(count)
    
    X = df.drop('class', axis=1)
    
    
    y = df['class']
    X = X.drop(X.index[len(X)-1])
    y = y.dropna()
    

    df_now = get_now_data(datetime.datetime.now(), period_days=test_days, money=moneyname, foot = FOOT, back_days = back_day)
    print("money: "+moneyname)
    
#     count = [0,0]
    #df['class'] = (df["Rate"]).apply(lambda x: classify(x, count, 0.001))
    #df_now['class'] = (df_now["Rate"]).apply(lambda x: classify_two(x, count))
    df_now['class'] = df_now['class'].shift(-1)
    print("NOW Data from" +str(test_days)+ "days ago")
    print(" LOW/MID/HIGH")
    print(count)
    
    print("learning data")
    print(df.head(1).index.values, end=' to ')
    print(df.tail(1).index.values)
    
    print("")
    print("test data")
    print(df_now.head(1).index.values,end=' to ')
    print(df_now.tail(1).index.values)
    print("")
    X_now = df_now.drop('class', axis=1)
    
    X_next = X_now.tail(1) #最終予測に利用
    y_now = df_now['class']
    X_now = X_now.drop(X_now.index[len(X_now)-1])
    y_now = y_now.dropna()
    
    X_train = X
    y_train = y
    X_test = X_now
    y_test = y_now
    
    
    names = [
#         "Nearest Neighbors",
#         "Linear SVM",
#         "RBF SVM",
#         "Decision Tree",
        "Random Forest",
#         "AdaBoost",
#         "Naive Bayes",
#         "Linear Discriminant Analysis",
#         "Quadratic Discriminant Analysis"
    ]
    classifiers = [
#     KNeighborsClassifier(3),
#     SVC(kernel="linear", C=0.025),
#     SVC(gamma=2, C=1),
#     DecisionTreeClassifier(max_depth=10),
    RandomForestClassifier(),#max_depth=10, n_estimators=10, max_features=1
#     AdaBoostClassifier(),
#     GaussianNB(),
#     LinearDiscriminantAnalysis(),
#     QuadraticDiscriminantAnalysis()
    ]
    
    accuracy_scores = []
    precision_scores = []
    next_class = []
    profit = []
    
    clearlist = []
    
    for name, clf in zip(names, classifiers):
        print(name)
        
        def randomforest_cv(n_estimators, min_samples_split, max_features):
            val = cross_val_score(
                RandomForestClassifier(
                    n_estimators=int(n_estimators),
                    min_samples_split=int(min_samples_split),
                    max_features=max_features,
                    random_state=0,
                ),
                X_train, y_train,
                scoring = 'accuracy',
                cv = 3, # 3-fold
                n_jobs = -1 # use all CPUs
            ).mean()
            return val

        randomforest_cv_bo = BayesianOptimization(
            randomforest_cv,
            {'n_estimators': (10, 250),
            'min_samples_split': (2, 25),
            'max_features': (0.1, 0.999),
            }
        )

        gp_params = {"alpha": 1e-5}
        randomforest_cv_bo.maximize(n_iter=50, **gp_params)
        print(randomforest_cv_bo.max)
    
        
        clf = RandomForestClassifier(
                    n_estimators=int(randomforest_cv_bo.max['params']['n_estimators']),
                    min_samples_split=int(randomforest_cv_bo.max['params']['min_samples_split']),
                    max_features=randomforest_cv_bo.max['params']['max_features'],
                    random_state=0,
                )
        
        clf.fit(X_train, y_train)
        result = clf.predict(X_test)*-1
        profit, values = calc_profit(result, y_test)
        precision = precision_score(y_test, result, average=None)
        accuracy = accuracy_score(y_test, result)
        confusion = confusion_matrix(y_test, result)
        
        print(clf.feature_importances_)
        
        if(1):
        #if(precision[0]!=1 and precision[1]!=1 and abs(confusion[0][0]-confusion[1][1])<(confusion[0][0]+confusion[1][1])/4):
        #if(precision[0]>0.4 and precision[1]>0.4):
#             score = clf.score(X_test, y_test)
            next_class.extend(clf.predict(X_next))

            print('accuracy_score: ', end='')
            print(accuracy_score(y_test, result))
            accuracy_scores.extend([accuracy_score(y_test, result)])
            print('confusion_matrix: ')
            print(confusion_matrix(y_test, result))
            print('precision_score: ', end='')
            print(precision_score(y_test, result, average=None))
            precision_scores.extend([precision_score(y_test, result, average=None)])

            print(str(len(result))+"hours trade")
            print("profit: "+str(profit))

            print('', flush=True)

            model_name = moneyname + "_" + name + "_" + str(datanum) +"_"+str(accuracy)+".sav"
            #clearlist.extend([model_name])
            print("CLEAR: " + model_name)
            pickle.dump(clf, open("models/"+model_name, 'wb'))

    #print(clearlist)
    return accuracy_scores, precision_scores, next_class, profit

def make_models(test_days, namelist, moneylist, back_days, datanum):
    for i in range(len(namelist)):
        print(namelist[i], flush=True)
        acc, pre, next_class, profit = predict(namelist[i], moneylist[i], test_days, back_days, datanum)
        print(acc)


def calc_profit(predict_result, true_data):
    win=0
    lose=0
    for i in range(len(predict_result)):
            if (predict_result[i]==-1):#Low予想のとき
                if (predict_result[i]==true_data[i]): #予測成功
                    win+=1
                else:
                    lose+=1
            elif(predict_result[i]==1):#High予想の時
                if (predict_result[i]==true_data[i]): #予測成功
                    win+=1
                else:
                    lose+=1
    profit = win*0.95-lose #利率を算出
    values = predict_result*profit #予測結果×利率により予測の価値を算出
    return profit, values

def plot_result(index, predict_result, true_data):
    plt.figure(figsize=(20,5))
    index = index.astype(float)

    color = []
    marker = []
    accuracy = []
    
    for x in range(0,len(predict_result)):
        if true_data.values[x] == predict_result[x]:
            color.extend(['blue'])
            accuracy.extend([1])
        
        elif predict_result[x]==0:
            color.extend(['green'])
            accuracy.extend([0])
        else:
            color.extend(['red'])
            accuracy.extend([-1])


    plt.scatter(index.index.values, index["Rate"].values, color=color)

    plt.xticks(rotation=90)
    plt.show()
    
    return accuracy


def ensemble(model_names, moneyname, days):
    

    global PAGE
    
    temp = PAGE
    #PAGE = trader.select_money(moneyname, temp)
    
    df_now = get_now_data(datetime.datetime.now(), period_days=days, money=moneyname, foot = FOOT, back_days = NIGHT)###########
    print("money: "+moneyname)
#     count = [0,0]
    
    #df_now['class'] = (df_now["Rate"]).apply(lambda x: classify_two(x, count))
    df_now['class'] = df_now['class'].shift(-1)
    print("NOW Data from" +str(days)+ "days ago")
    print(" LOW/MID/HIGH")
    print(count)

    print("")
    print("test data")
    print(df_now.head(1).index.values,end=' to ')
    print(df_now.tail(1).index.values)
    print("")
    X_now = df_now.drop('class', axis=1)
    
    X_next = X_now.tail(1) #最終予測に利用
    y_now = df_now['class']
    X_now = X_now.drop(X_now.index[len(X_now)-1])
    y_now = y_now.dropna()
    
    values = []
    values_sum = []
    win=0
    lose=0
    answer = 0
    
    #add DAY/2 前半でprecisionを作成、後半で実際の的中率を検討 ver3.1
#     size = len(X_now)
#     X_test = X_now.tail(int(size/2)-1)
#     X_now = X_now.head(int(size/2))
    
#     y_test = y_now.tail(int(size/2)-1)
#     y_now = y_now.head(int(size/2))
    #print(len(X_test), len(y_test), len(X_now), len(y_now))
    
    
    for x in range(len(model_names)):
        clf = pickle.load(open(model_names[x], 'rb')) #clfをload
        print(model_names[x], flush=True)
        #result = clf.predict(X_now) #predict
        #precision = precision_score(y_now, result, average=None)
        #if(precision[0]!=0 and precision[1]!=0 and precision[0]!=1 and precision[1]!=1):
        if(1):
       
            #score表示
#             print("accuracy_score: ", end="")
#             accuracy = accuracy_score(y_now, result)
#             print(accuracy)
#             print("precision_score: ", end="")
#             print(precision)
#             print(confusion_matrix(y_now, result))

#             profit, values = calc_profit(result, y_now) #予測結果について利率と、予測*利率による予測の信頼性を算出
            
            
            #answer += clf.predict(X_next)[0]*profit
            answer += clf.predict(X_next)[0]
            
            result = clf.predict(X_now)
             #predict
            
            ####valuesの算出方法で予測精度が決まる
            
            #values = result*accuracy #profit:-16
            
            #values = np.where(result<0, result*precision[0], result*precision[1]) #profit:-14
            #values = result  #profit:-18
            accuracy = accuracy_score(y_now, result)
            print(accuracy)
            values = -result#*(1 - int(accuracy*100%2)*2)
            
#             plot_result(X_now, values, y_now)
            
#             print("profit: ",end='')
#             profit, valuess = calc_profit(values, y_now)
#             print(profit)
            
#             R=0.3  #R=0.37で-13.0,  R=0.35で-16 R=0.45で15, R=0.5で-15
        
#             if(precision[0]<R):
#                 precision[0] = 0
#             if(precision[1]<R):
#                 precision[1] = 0
#             values = np.where(result<0, result*precision[0], result*precision[1])

            
            #values = np.where(result<0, result*(((precision[0])*100)**2), (result*(((precision[1])*100)**2)))
            
            if len(values_sum)==0:#1回目は配列をコピー
                values_sum = values
            else:
                values_sum += values #２回目からは加算
            print(values)
    
        #plot_result(X_now, result, y_now)
    if(answer>0):
        print("HIGH")
        hl = "HIGH"
        #trader.high_entry()
        
    else:
        print("LOW")
        hl = "LOW"
        #trader.low_entry()    
    print(values_sum)
    
    if(values_sum==[]):
        print("No model")
        return 0
    
    ensemble_result = np.where(values_sum<0, -1, 1) #クラス分類
    
    print("")
    print("ensemble_result: ",end='')
    print(accuracy_score(y_now.values, ensemble_result))
    print(ensemble_result)
    print(y_now.values)
    print(values_sum)
    print("precision_score: ", end="")
    print(precision_score(y_now, ensemble_result, average=None))
    print('confusion_matrix: ')
    print(confusion_matrix(y_now, ensemble_result))
    print("profit: ",end='')
    profit, values = calc_profit(ensemble_result, y_now)
    print(profit)
    print("next: ",end='', flush=True)
    print(answer)
    
    print(X_next.head(1))
    accuracy = plot_result(X_now, ensemble_result, y_now)
    
    global profit_sum
    profit_sum += profit
    
    global RESULT
    RESULT.extend([moneyname+": "+hl])
    return accuracy
        
    
def compere_ensemble(models_array, moneys_array, days):
    results = []
    
    for i in range(len(moneys_array)):
        results.append(np.array(ensemble(models_array[i], moneys_array[i], days)).astype('int'))
    
    np.savetxt('out.csv',results,delimiter=',')
    print(results)
    
def ensemble_set(models_array, moneys_array, days):
    
    for i in range(len(models_array)):
        print(ensemble(models_array[i], moneys_array[i], days))    

def plot_rates(hours, moneys):
    
    result = []
    accord = []
    plt.figure(figsize=(40,20))
    for i in range(len(moneys)):
#         df_now = get_now_data(datetime.datetime.now(), period_Days=days, money=moneys[i], foot = 'D', back_days = 0)
        df_now = get_now_data(datetime.datetime.now(), period_days=days, money=moneys[i], foot = FOOT, back_days = 0)
        plt.plot(df_now.index.values, df_now["Rate"].values)
        count = [0,0]
        #df_now['class'] = (df_now["Rate"]).apply(lambda x: classify_two(x, count))
        result.append(df_now["class"].values)
    
    win = 0
    lose = 0
    
    for l in range(len(result[0])):
        temp = 0
        for m in range(len(moneys)):
            if result[m][l]==1:
                temp+=1
        if(temp==len(moneys)/2):
            accord.extend([-1])
            lose += len(moneys)/2
            win += len(moneys)/2
        else:
            accord.extend([temp])
            win += max(temp, len(moneys)-temp)
            lose += min(temp, len(moneys)-temp)
    
    
    plt.xticks(rotation=90)
    plt.hlines([0], 0, days, "blue", linestyles='dashed')     # hlines
    plt.show()
    print(accord)
    
    pay = 1000
    pay_sum = pay*len(moneys)
    
    print("win: ", win)
    print("lose: ", lose)
    print("rate: ", win/(win+lose))
    print("profit: ", (win*0.95-lose)*pay_sum)

COUNT = 0
PAGE = 0
    
def job():
    global COUNT
    
    time.sleep(5)
    print("START")
    
    if COUNT == 0:
        print("Set up browser...")
        trader.setup_demo()
        time.sleep(3)
    else:
        print("Ready for entry...")
        time.sleep(13)
    main()
    COUNT += 1
    
    #print(RESULT)
    print('')
    print("profit_sum: ", end='')
    print(profit_sum)
    RESULT = []
#     print("Rebuild models...")
#     get_csv_and_make_models()
    print('')
    print('finished: ', end='')
    print(datetime.datetime.now())
    print('waiting...', flush=True)


    
    
def main():
    
    models = [glob.glob('models/USD_JPY*.sav'), glob.glob('models/EUR_JPY*.sav'), glob.glob('models/AUD_JPY*.sav'),
              glob.glob('models/GBP_JPY*.sav'), glob.glob('models/NZD_JPY*.sav'), glob.glob('models/CAD_JPY*.sav'),
              glob.glob('models/CHF_JPY*.sav')]#, glob.glob('ZAR_JPY*.sav')]
    moneys = ['USD_JPY', 'EUR_JPY', 'AUD_JPY', 'GBP_JPY', 'NZD_JPY', 'CAD_JPY', 'CHF_JPY']#, 'ZAR_JPY']
    
    
    #models = [glob.glob('models/USD_JPY*.sav'), glob.glob('models/EUR_USD*.sav'), glob.glob('models/EUR_JPY*.sav'), 
#               glob.glob('models/GBP_JPY*.sav'), glob.glob('models/AUD_JPY*.sav')]
    #moneylist = ['USD_JPY', 'EUR_USD', 'EUR_JPY', 'GBP_JPY', 'AUD_JPY']
    #ensemble(CHF_models, "CHF_JPY", 60)   #選択した通貨に対してアンサンブル
    #compere_ensemble(models, moneys, 30)   #それぞれの通貨に対してアンサンブルした結果をcsvに返す
    #plot_rates(30,moneys)
    
    ensemble_set(models, moneys, DAYS)  #指定したmoneyで結果を出す
    print(RESULT)

    print('')
    print("profit_sum: ", end='')
    print(profit_sum)
    
def get_csv_and_make_models():
    
    shutil.rmtree('csv')
    shutil.rmtree('models')
    
    os.mkdir('csv')
    os.mkdir('models')
    
    moneylist = ['USD_JPY', 'EUR_JPY', 'AUD_JPY', 'GBP_JPY', 'NZD_JPY', 'CAD_JPY', 'CHF_JPY']#, 'ZAR_JPY']
    #moneylist = ['USD_JPY', 'EUR_USD', 'EUR_JPY', 'GBP_JPY', 'AUD_JPY']
    
#     array50 = get_moneys_csv(back_days=DAYS, days=50, moneylist = moneylist)
#     array100 = get_moneys_csv(back_days=DAYS*2, days=100, moneylist = moneylist)
    array500 = get_moneys_csv(back_days=DAYS*2, days=500, moneylist = moneylist) # 指定期間のcsvをsave
    array1000 = get_moneys_csv(back_days=DAYS*2, days=1000, moneylist = moneylist)
    array1500 = get_moneys_csv(back_days=DAYS*2, days=1500, moneylist = moneylist)
    array2000 = get_moneys_csv(back_days=DAYS*2, days=2000, moneylist = moneylist)
    array2500 = get_moneys_csv(back_days=DAYS*2, days=2500, moneylist = moneylist)
    array3000 = get_moneys_csv(back_days=DAYS*2, days=3000, moneylist = moneylist)
    
   #make_models(test_days, namelist, moneylist, back_days, datanum):
    
#     make_models(DAYS, array50, moneylist, 0, 50)
#     make_models(DAYS, array100, moneylist, 0, 100)
    make_models((DAYS), array500, moneylist, DAYS, 500)
    make_models((DAYS), array1000, moneylist, DAYS, 1000)
    make_models((DAYS), array1500, moneylist, DAYS, 1500)
    make_models((DAYS), array2000, moneylist, DAYS, 2000)
    make_models((DAYS), array2500, moneylist, DAYS, 2500)
    make_models((DAYS), array3000, moneylist, DAYS, 3000)

NIGHT = 0
FOOT = 'D'

DAYS = 100

PAST = 0

count = [0,0]

get_csv_and_make_models()
RESULT = []

profit_sum = 0

# now = datetime.datetime.now()
# start_time = datetime.datetime(now.year, now.month, now.day, now.hour+1, 0, 0)
# print((start_time-now).total_seconds()-10)
# time.sleep((start_time-now).total_seconds()-10)


#import trader
#trader.setup_demo()
main()
#COUNT += 1
# print(RESULT)
# print('')
# print("profit_sum: ", end='')
# print(profit_sum)
# print("pay 1000: ", end=str(profit_sum*1000))

#schedule.every(60*59).seconds.do(job)

print('Ready... :', end='')
print(datetime.datetime.now())
print("Did you restart the kernel?")

def auto():
    schedule.every().day.at("07:00").do(job)

    while True:
        schedule.run_pending()
        time.sleep(5)

# print(RESULT)

# print('')
# print("profit_sum: ", end='')
# print(profit_sum)

print("pay 1000: ", end=str(profit_sum*1000))




"USD_JPY_2019-01-20_2020-06-03_500.csv", "EUR_JPY_2019-01-20_2020-06-03_500.csv", "AUD_JPY_2019-01-20_2020-06-03_500.csv", "GBP_JPY_2019-01-20_2020-06-03_500.csv", "NZD_JPY_2019-01-20_2020-06-03_500.csv", "CAD_JPY_2019-01-20_2020-06-03_500.csv", "CHF_JPY_2019-01-20_2020-06-03_500.csv", "USD_JPY_2017-09-07_2020-06-03_1000.csv", "EUR_JPY_2017-09-07_2020-06-03_1000.csv", "AUD_JPY_2017-09-07_2020-06-03_1000.csv", "GBP_JPY_2017-09-07_2020-06-03_1000.csv", "NZD_JPY_2017-09-07_2020-06-03_1000.csv", "CAD_JPY_2017-09-07_2020-06-03_1000.csv", "CHF_JPY_2017-09-07_2020-06-03_1000.csv", "USD_JPY_2016-04-25_2020-06-03_1500.csv", "EUR_JPY_2016-04-25_2020-06-03_1500.csv", "AUD_JPY_2016-04-25_2020-06-03_1500.csv", "GBP_JPY_2016-04-25_2020-06-03_1500.csv", "NZD_JPY_2016-04-25_2020-06-03_1500.csv", "CAD_JPY_2016-04-25_2020-06-03_1500.csv", "CHF_JPY_2016-04-25_2020-06-03_1500.csv", "USD_JPY_2014-12-12_2020-06-03_2000.csv", "EUR_JPY_2014-12-12_2020-06-03_2000.csv", "AUD_JPY_2014-12-12_2020-06-03_2000.csv",